## 0. Import packages

In [1]:
import pandas as pd
import gzip
import math
from tqdm import tqdm
from itertools import chain, combinations
tqdm.pandas() #for progres_apply etc.

## 1. Load data from previous step

In [2]:
import pickle
train_df = pickle.load(open("pickle_dumps/train_df.p", "rb"))

In [3]:
train_df

,user_id,item_id,recommend
0,302211,"[1971, 4158, 13725]","[True, True, True]"
1,1378215,[13884],[True]
2,403149,[2678],[True]
3,1170243,"[10819, 12169, 13215]","[True, True, True]"
4,127771,"[703, 1241, 2098, 2382, 8908, 9676, 13637, 13997]","[True, True, True, True, True, True, True, True]"
...,...,...,...
1188484,166817,"[440, 8549]","[True, True]"
1188485,694962,[5266],[True]
1188486,34608,"[284, 11238, 12663]","[True, True, True]"
1188487,172968,[440],[True]


In [4]:
data = data = train_df.set_index("user_id").to_dict()["item_id"]

In [5]:

def eclat(prefix, items):
    frequents = []
    while items:
        i,itids = items.pop()
        isupp = len(itids)
        if isupp >= minsup:
            frequents.append((frozenset(prefix + [i]), isupp))
            suffix = [] 
            for j, ojtids in items:
                jtids = set(itids) & set(ojtids)
                if len(jtids) >= minsup:
                    suffix.append((j,jtids))
            frequents.extend(eclat(prefix+[i], sorted(suffix, key=lambda item: len(item[1]), reverse=True)))
    return frequents

In [ ]:
%%time

minsup = 5

frequent_itemsets = eclat([], sorted(data.items(), key=lambda item: len(item[1]), reverse=True))


In [ ]:
print(len(frequent_itemsets))

In [ ]:
def subsets(itemset, include_empty_set=False):
    """ List all strict subsets of an itemset without the empty set or with the empty set if include_empty_set=True
        subsets({1,2,3}) --> [{1}, {2}, {3}, {1, 2}, {1, 3}, {2, 3}]
    """
    s = list(itemset)
    if include_empty_set:
        return map(set, chain.from_iterable(combinations(s, r) for r in range(0, len(s) + 1)))
    return map(set, chain.from_iterable(combinations(s, r) for r in range(1, len(s))))

In [ ]:
def deriveRules(itemsets, minconf):
    """ Returns all rules with conf >= minconf that can be derived from the itemsets.
        Return: list of association rules in the format: [(antecedent, consequent, supp, conf), ...]
    """
    search_items = dict(itemsets)
    rules = []
    for i, (item_set, supp) in enumerate(itemsets):
        if len(item_set) > 1:
            for subset in subsets(item_set):  # for each subset generate a rule
                antecedent = frozenset(subset)
                consequent = item_set - subset
                if len(consequent) == 1:
                    conf = supp / search_items[antecedent]
                    if conf >= minconf:
                        rules.append((antecedent, consequent, supp, conf))

    return rules

rules = deriveRules(frequent_itemsets, 0.8)

In [ ]:
rules

In [ ]:
len(rules)